In [1]:
import collections
import pandas as pd
import pyreadr
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from venn import venn
import numpy as np
import math
import scipy

/var/folders/y8/66w8_8kj0dx2znq80cdbl_rw0000gn/T/ipykernel_80736/2185171821.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
clinical_df = pyreadr.read_r('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/figurecode/data/20221109_TRACERx421_all_patient_df.rds')
clinical_df = pd.DataFrame(clinical_df[None])
clinical_df.head()

,cruk_id,tumour_id_muttable_cruk,tumour_id_per_patient,age,sex,ethnicity,cigs_perday,years_smoking,packyears,smoking_status_merged,...,os_time,cens_dfs,dfs_time,cens_dfs_any_event,dfs_time_any_event,cens_lung_event,lung_event_time,Relapse_cat,Relapse_cat_new,tx100
0,CRUK0034,CRUK0034,CRUK0034,68,Female,White- Irish,20.0,35,35.000,Ex-Smoker,...,1849,0,1849,0,1849,0,1849,NaN,No rec,True
1,CRUK0150,CRUK0150,CRUK0150,81,Male,White- British,44.5,49,109.025,Ex-Smoker,...,1362,1,1362,1,1362,0,1362,NaN,No rec,False
2,CRUK0159,CRUK0159,CRUK0159,60,Male,White- British,20.0,38,38.000,Smoker,...,2224,1,2224,1,1935,0,2224,Second primary lung,No rec,False
3,CRUK0090,CRUK0090,CRUK0090,65,Male,White- British,10.0,35,17.500,Ex-Smoker,...,2365,1,1143,1,1143,1,1143,Intrathoracic,Intrathoracic,True
4,CRUK0045,CRUK0045,CRUK0045,85,Male,White- British,10.0,25,12.500,Ex-Smoker,...,986,1,917,1,917,1,917,Intrathoracic,Intrathoracic,True


In [4]:
tumor_df = pyreadr.read_r('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/LCCE_converted_data/20221109_TRACERx421_all_tumour_df.rds')
tumor_df = pd.DataFrame(tumor_df[None])
tumor_df.head()

,tumour_id,patient_id,clinical_sex,age,ethnicity,Histology_per_tumour_id_muttable,histology_3,LUAD_pred_subtype_with.IMA_per_tumour,site_per_lesion,pT_stage_per_lesion,pN_stage_per_lesion,size_pathology_per_lesion,vascular_invasion_per_lesion,pleural_invasion_per_lesion,smoking_status_merged,cigs_perday,years_smoking,pack_years,is.family.lung,pathologyTNM
0,CRUK0034,CRUK0034,Female,68,White- Irish,Invasive adenocarcinoma,LUAD,acinar,Right Upper Lobe,2a,0,40,Yes,No,Ex-Smoker,20.0,35,35.000,True,IB
1,CRUK0150,CRUK0150,Male,81,White- British,LCNEC,Other,NaN,Left Upper Lobe,3,0,75,Yes,No,Ex-Smoker,44.5,49,109.025,False,IIB
2,CRUK0159,CRUK0159,Male,60,White- British,Invasive adenocarcinoma,LUAD,acinar,Right Upper Lobe,2a,0,36,Yes,Yes,Smoker,20.0,38,38.000,False,IB
3,CRUK0090,CRUK0090,Male,65,White- British,Squamous cell carcinoma,LUSC,NaN,Right Upper Lobe,2a,1,35,Yes,Yes,Ex-Smoker,10.0,35,17.500,False,IIA
4,CRUK0045,CRUK0045,Male,85,White- British,Invasive adenocarcinoma,LUAD,papillary,Left Lower Lobe,2a,0,48,Yes,Yes,Ex-Smoker,10.0,25,12.500,False,IB


In [5]:
mut_table = pyreadr.read_r('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/LCCE_converted_data/20221109_TRACERx421_mutation_table.rds')
mut_table = pd.DataFrame(mut_table[None])
mut_table.head()

,patient_id,tumour_id,mutation_id,chr,start,stop,ref,var,Hugo_Symbol,func,...,AAChange,GL_VAF,GL_nAlt,GL_depth,ITHState,PyCloneClonal_SC,PyCloneCluster_SC,cleanCluster_SC,combTiming_SC,DriverMut
0,CRUK0005,CRUK0005,CRUK0005:19:34291428:G,chr19,34291428,34291428,G,T,KCTD15,intronic,...,NaN,0,0,187,1,S,3.0,True,subclonal,False
1,CRUK0005,CRUK0005,CRUK0005:19:9084770:C,chr19,9084770,9084770,C,A,MUC16,exonic,...,p.V2349L,0.39,2,510,1,C,2.0,True,early,False
2,CRUK0005,CRUK0005,CRUK0005:1:2160418:G,chr1,2160418,2160418,G,T,SKI,exonic,...,p.P71P,0,0,114,1,C,2.0,True,early,False
3,CRUK0005,CRUK0005,CRUK0005:19:40095981:G,chr19,40095981,40095981,G,T,LGALS13,exonic,...,p.G86C,0,0,659,1,C,2.0,True,early,False
4,CRUK0005,CRUK0005,CRUK0005:19:41313703:C,chr19,41313703,41313703,C,G,RAB4B-EGLN2,ncRNA_intronic,...,NaN,0,0,894,3,S,5.0,True,subclonal,False


In [6]:
clinical_df = clinical_df.sort_values(by=['cruk_id'])
clinical_df

,cruk_id,tumour_id_muttable_cruk,tumour_id_per_patient,age,sex,ethnicity,cigs_perday,years_smoking,packyears,smoking_status_merged,...,os_time,cens_dfs,dfs_time,cens_dfs_any_event,dfs_time_any_event,cens_lung_event,lung_event_time,Relapse_cat,Relapse_cat_new,tx100
88,CRUK0001,CRUK0001,CRUK0001,84,Male,White- British,20.0,30,30.00,Ex-Smoker,...,1120,1,1120,1,1120,0,1120,NaN,No rec,True
79,CRUK0002,CRUK0002,CRUK0002,81,Male,White- British,5.0,35,8.75,Ex-Smoker,...,1901,0,1901,0,1901,0,1901,NaN,No rec,True
26,CRUK0003,CRUK0003,CRUK0003,78,Female,White- British,0.0,0,0.00,Never Smoked,...,2304,1,217,1,217,1,217,Intra & Extra,Intra & Extra,True
23,CRUK0004,CRUK0004,CRUK0004,69,Female,White- British,10.0,27,13.50,Ex-Smoker,...,1323,1,325,1,325,1,325,Intra & Extra,Intra & Extra,True
7,CRUK0005,CRUK0005,CRUK0005,74,Female,White- British,20.0,56,56.00,Ex-Smoker,...,649,1,649,1,649,0,649,NaN,No rec,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,CRUK0861,CRUK0861,CRUK0861,65,Female,Middle eastern,10.0,45,22.50,Smoker,...,1084,0,1084,0,1084,0,1084,NaN,No rec,False
418,CRUK0864,CRUK0864,CRUK0864,78,Female,White- British,10.0,30,15.00,Ex-Smoker,...,1434,0,1434,0,1434,0,1434,NaN,No rec,False
416,CRUK0872,CRUK0872,CRUK0872,66,Male,White- British,20.0,54,54.00,Ex-Smoker,...,893,1,883,1,883,1,883,Intra & Extra,Intra & Extra,False
420,CRUK0881,CRUK0881_Tumour1,CRUK0881_Tumour1;CRUK0881_Tumour2,84,Male,White- British,20.0,70,70.00,Smoker,...,758,1,748,1,748,1,748,Intrathoracic,Intrathoracic,False


In [7]:
# Making the training data

y_train = []
for index, row in clinical_df.iterrows():
    if row['first_dfs_any_event_rec.or.new.primary'] == 'recurrence':
        y_train.append(1)
    else:
        y_train.append(0)

y = np.array(y_train)
print(y)

[0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 1 0 0 1 1 1 1
 1 0 1 0 0 1 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 1 0
 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1
 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 1
 1 0 0 0 0 0 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0
 0 0 0 1 1 0 1 0 1 0 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0
 1 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 1
 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 1 0 0 1 1 1 0 1
 0 0 0 1 0 0 0 1 1 1 0 0 1 0 1 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 0
 1 0 0 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1
 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 1
 0 1 0 0 0 0 0 1 1 0 0 1 1 0]


In [2]:
gene_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/100_gene_matrix.csv')
driver_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/100_driver_matrix.csv')
fifty_gene_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/50_gene_matrix.csv')
fifty_driver_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/50_driver_matrix.csv')
twenty_gene_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/20_gene_matrix.csv')
twenty_driver_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/20_driver_matrix.csv')
ten_driver_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/10_driver_matrix.csv')

In [3]:
gene_matrix = gene_matrix.set_index(gene_matrix.columns[0])
driver_matrix = driver_matrix.set_index(driver_matrix.columns[0])
fifty_gene_matrix = fifty_gene_matrix.set_index(fifty_gene_matrix.columns[0])
fifty_driver_matrix = fifty_driver_matrix.set_index(fifty_driver_matrix.columns[0])
twenty_gene_matrix = twenty_gene_matrix.set_index(twenty_gene_matrix.columns[0])
twenty_driver_matrix = twenty_driver_matrix.set_index(twenty_driver_matrix.columns[0])
ten_driver_matrix = ten_driver_matrix.set_index(ten_driver_matrix.columns[0])

In [5]:
gene_matrix

,ABCA13,ADAMTS12,ADAMTS16,ADAMTS20,AFF2,ANK2,APOB,ASTN1,BAI3,CACNA1B,...,TRY2P,TTC40,TTN,UNC79,UNC80,USH2A,WDFY4,XIRP2,ZFHX4,ZNF536
patient_id,,,,,,,,,,,,,,,,,,,,,
CRUK0001,1,1,0,1,1,1,1,0,1,0,...,1,0,1,0,1,1,1,0,0,1
CRUK0002,0,0,0,1,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
CRUK0003,0,0,1,0,0,0,0,0,0,1,...,0,0,1,1,0,1,0,0,0,0
CRUK0004,1,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
CRUK0005,1,1,0,1,1,1,1,0,0,0,...,1,0,1,0,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CRUK0861,0,0,0,0,1,1,1,0,1,0,...,0,0,1,0,0,0,0,0,0,0
CRUK0864,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
CRUK0872,0,0,0,0,0,0,1,1,0,0,...,0,0,1,0,1,1,0,0,1,0


In [6]:
sum_100_genes = sum(sum(row) for row in gene_matrix.values)
sum_100_genes

15334

In [7]:
sum_100_drivers = sum(sum(row) for row in driver_matrix.values)
sum_100_drivers

2162

In [8]:
sum_20_genes = sum(sum(row) for row in twenty_gene_matrix.values)
sum_20_genes

4551

In [9]:
sum_20_drivers = sum(sum(row) for row in twenty_driver_matrix.values)
sum_20_drivers

1065

# Use value_counts to get the frequency of each gene
gene_counts = mut_table['Hugo_Symbol'].value_counts()

# Get the top 100 most common genes
top_100_genes = gene_counts.head(100)

# Convert the result to a list of tuples (gene, frequency)
gene_frequency_list = list(zip(top_100_genes.index, top_100_genes.values))

print(gene_frequency_list)

gene_matrix = gene_matrix.sort_values(by=['patient_id'])
driver_matrix = driver_matrix.sort_values(by=['patient_id'])

gene_matrix.to_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/100_gene_matrix.csv')
driver_matrix.to_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/100_driver_matrix.csv')

# Extract the gene names from the gene_frequency_list
hundred_genes = [gene for gene, _ in gene_frequency_list]

# Filter the original DataFrame based on the selected genes
filtered_mut_table = mut_table[mut_table['Hugo_Symbol'].isin(hundred_genes)]

# Create a binary matrix (presence or absence of genes for each patient)
gene_matrix = pd.crosstab(filtered_mut_table['patient_id'], filtered_mut_table['Hugo_Symbol']).clip(upper=1)

# Display the resulting gene matrix
print(gene_matrix.shape)
print(gene_matrix)

In [10]:
# predicting recurrence with absence/presence of 100 most commonly mutated genes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV


X_train, X_test, y_train, y_test = train_test_split(gene_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

lr_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(lr_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 100) (169, 100) (252,) (169,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

Best parameters:  {'C': 0.1}
Best score:  0.5107843137254902
Predictions:  [0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1
 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5475407268170426
ROC AUC:  0.5238095238095237
               precision    recall  f1-score   support

No Recurrence       0.69      0.92      0.79       112
   Recurrence       0.53      0.18      0.26     

from sklearn.metrics import roc_curve, roc_auc_score
y_score = lr_model.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_score, pos_label=1)
roc_auc = roc_auc_score(y_test, y_score)

plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate or (1 - Specifity)')
plt.ylabel('True Positive Rate or (Sensitivity)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

from sklearn.metrics import roc_curve, roc_auc_score
y_score = lr_model.predict_proba(gene_matrix)[:, 1]
fpr, tpr, thresholds = roc_curve(y, y_score, pos_label=1)
roc_auc = roc_auc_score(y, y_score)

plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate or (1 - Specifity)')
plt.ylabel('True Positive Rate or (Sensitivity)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [11]:
from sklearn import svm

X_train, X_test, y_train, y_test = train_test_split(gene_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

svm_model = svm.SVC(probability=True)

grid_search = GridSearchCV(svm_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 100) (169, 100) (252,) (169,)
Best parameters:  {'C': 10}
Best score:  0.551247771836007
Predictions:  [0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0
 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 1 0 1 0
 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5515350877192983
ROC AUC:  0.5277255639097744
               precision    recall  f1-score   support

No Recurrence       0.69      0.88      0.77       112
   Recurrence  

In [13]:
# Neural network/Multi-layer perceptron (MLP) algorithm

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(gene_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 100) (169, 100) (252,) (169,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network

Best parameters:  {'alpha': 0.01, 'hidden_layer_sizes': (1,)}
Best score:  0.5325311942959001
Predictions:  [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 0 1 1
 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.525375939849624
ROC AUC:  0.5034461152882206
               precision    recall  f1-score   support

No Recurrence       0.68      0.89      0.77       112
   Recurrence    

fpr, tpr, thresholds = roc_curve(y_test, y_score, pos_label=1)

plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate or (1 - Specifity)')
plt.ylabel('True Positive Rate or (Sensitivity)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Use boolean indexing to filter rows where 'DriverMut' is True
driver_table = mut_table[mut_table['DriverMut']]

# Count the occurrences of each gene in the filtered DataFrame
driver_counts = driver_table['Hugo_Symbol'].value_counts()

# Get the top 100 most common genes
top_100_drivers = driver_counts.head(100)

# Convert the result to a list of tuples (gene, frequency)
driver_frequency_list = list(zip(top_100_drivers.index, top_100_drivers.values))

print(driver_frequency_list)

# Extract the gene names from the driver_frequency_list
hundred_drivers = [gene for gene, _ in driver_frequency_list]

# Filter the original DataFrame based on the selected genes
filtered_drivers_table = mut_table[mut_table['Hugo_Symbol'].isin(hundred_drivers)]

# Create a binary matrix based on both gene presence/absence and 'DriverMut' values
driver_matrix = pd.crosstab(filtered_drivers_table['patient_id'], filtered_drivers_table['Hugo_Symbol'], values=filtered_drivers_table['DriverMut'].astype(int), aggfunc='max').clip(upper=1).fillna(0)

driver_matrix = driver_matrix.astype(int)

# Display the resulting gene matrix
print(driver_matrix.shape)
print(driver_matrix)

driver_matrix.sum(axis=0).sort_values(ascending=False)
# this is different from driver_frequency_list because repeated driver mutations in the same gene are only once in the matrix

In [55]:
# predicting recurrence with absence/presence of 100 most common driver mutations

X_train, X_test, y_train, y_test = train_test_split(driver_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

lr_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(lr_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 100) (169, 100) (252,) (169,)


Best parameters:  {'C': 1}
Best score:  0.5043672014260249
Predictions:  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0
 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 1 0 1 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5072055137844611
ROC AUC:  0.47485902255639095
               precision    recall  f1-score   support

No Recurrence       0.67      0.82      0.74       112
   Recurrence       0.35      0.19      0.25      

In [56]:

X_train, X_test, y_train, y_test = train_test_split(driver_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

svm_model = svm.SVC(probability=True)

grid_search = GridSearchCV(svm_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 100) (169, 100) (252,) (169,)
Best parameters:  {'C': 1e-05}
Best score:  0.5
Predictions:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5
ROC AUC:  0.5514567669172932
               precision    recall  f1-score   support

No Recurrence       0.66      1.00      0.80       112
   Recurrence       0.00      0.00      0

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [57]:
# Neural network/Multi-layer perceptron (MLP) algorithm

X_train, X_test, y_train, y_test = train_test_split(driver_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 100) (169, 100) (252,) (169,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py

Best parameters:  {'alpha': 0.01, 'hidden_layer_sizes': (2,)}
Best score:  0.5212121212121212
Predictions:  [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5167606516290727
ROC AUC:  0.5029761904761905
               precision    recall  f1-score   support

No Recurrence       0.67      0.91      0.77       112
   Recurrence   

# Extract the gene names from the gene_frequency_list
fifty_genes = [gene for gene, _ in gene_frequency_list[:50]]

# Filter the original DataFrame based on the selected genes
filtered_mut_table = mut_table[mut_table['Hugo_Symbol'].isin(fifty_genes)]

# Create a binary matrix (presence or absence of genes for each patient)
fifty_gene_matrix = pd.crosstab(filtered_mut_table['patient_id'], filtered_mut_table['Hugo_Symbol']).clip(upper=1)

# Identify missing patients
all_patients = set(mut_table['patient_id'])
patients_with_genes = set(filtered_mut_table['patient_id'])
missing_patients = all_patients - patients_with_genes

# Add missing patients to the binary matrix with all 0s
for missing_patient in missing_patients:
    fifty_gene_matrix.loc[missing_patient, :] = 0

# Explicitly set the data type of the entire DataFrame to integers
fifty_gene_matrix = fifty_gene_matrix.astype(int)

# Display the resulting gene matrix
print(fifty_gene_matrix.shape)
print(fifty_gene_matrix)

# Extract the gene names from the gene_frequency_list
twenty_genes = [gene for gene, _ in gene_frequency_list[:20]]

# Filter the original DataFrame based on the selected genes
filtered_mut_table = mut_table[mut_table['Hugo_Symbol'].isin(twenty_genes)]

# Create a binary matrix (presence or absence of genes for each patient)
twenty_gene_matrix = pd.crosstab(filtered_mut_table['patient_id'], filtered_mut_table['Hugo_Symbol']).clip(upper=1)

# Identify missing patients
all_patients = set(mut_table['patient_id'])
patients_with_genes = set(filtered_mut_table['patient_id'])
missing_patients = all_patients - patients_with_genes

# Add missing patients to the binary matrix with all 0s
for missing_patient in missing_patients:
    twenty_gene_matrix.loc[missing_patient, :] = 0

# Explicitly set the data type of the entire DataFrame to integers
twenty_gene_matrix = twenty_gene_matrix.astype(int)

# Display the resulting gene matrix
print(twenty_gene_matrix.shape)
print(twenty_gene_matrix)

# Extract the gene names from the driver_frequency_list
fifty_drivers = [gene for gene, _ in driver_frequency_list[:50]]

# Filter the original DataFrame based on the selected genes
filtered_drivers_table = mut_table[mut_table['Hugo_Symbol'].isin(fifty_drivers)]

# Create a binary matrix based on both gene presence/absence and 'DriverMut' values
fifty_driver_matrix = pd.crosstab(filtered_drivers_table['patient_id'], filtered_drivers_table['Hugo_Symbol'], values=filtered_drivers_table['DriverMut'].astype(int), aggfunc='max').clip(upper=1).fillna(0)

# Identify missing patients
all_patients = set(mut_table['patient_id'])
patients_with_genes = set(filtered_drivers_table['patient_id'])
missing_patients = all_patients - patients_with_genes

# Add missing patients to the binary matrix with all 0s
for missing_patient in missing_patients:
    fifty_driver_matrix.loc[missing_patient, :] = 0

# Explicitly set the data type of the entire DataFrame to integers
fifty_driver_matrix = fifty_driver_matrix.astype(int)

# Display the resulting gene matrix
print(fifty_driver_matrix.shape)
print(fifty_driver_matrix)

# Extract the gene names from the driver_frequency_list
twenty_drivers = [gene for gene, _ in driver_frequency_list[:20]]

# Filter the original DataFrame based on the selected genes
filtered_drivers_table = mut_table[mut_table['Hugo_Symbol'].isin(twenty_drivers)]

# Create a binary matrix based on both gene presence/absence and 'DriverMut' values
twenty_driver_matrix = pd.crosstab(filtered_drivers_table['patient_id'], filtered_drivers_table['Hugo_Symbol'], values=filtered_drivers_table['DriverMut'].astype(int), aggfunc='max').clip(upper=1).fillna(0)

# Identify missing patients
all_patients = set(mut_table['patient_id'])
patients_with_genes = set(filtered_drivers_table['patient_id'])
missing_patients = all_patients - patients_with_genes

# Add missing patients to the binary matrix with all 0s
for missing_patient in missing_patients:
    twenty_driver_matrix.loc[missing_patient, :] = 0

# Explicitly set the data type of the entire DataFrame to integers
twenty_driver_matrix = twenty_driver_matrix.astype(int)

# Display the resulting gene matrix
print(twenty_driver_matrix.shape)
print(twenty_driver_matrix)

In [58]:
# predicting recurrence with absence/presence of 50 most commonly mutated genes

X_train, X_test, y_train, y_test = train_test_split(fifty_gene_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

lr_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(lr_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 50) (169, 50) (252,) (169,)


Best parameters:  {'C': 1e-05}
Best score:  0.5
Predictions:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5
ROC AUC:  0.4843358395989975
               precision    recall  f1-score   support

No Recurrence       0.66      1.00      0.80       112
   Recurrence       0.00      0.00      0.00        57

     accuracy        

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [59]:
# predicting recurrence with absence/presence of 20 most commonly mutated genes

X_train, X_test, y_train, y_test = train_test_split(twenty_gene_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

lr_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(lr_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 20) (169, 20) (252,) (169,)
Best parameters:  {'C': 1}
Best score:  0.5046345811051693
Predictions:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0
 0 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.47736528822055135
ROC AUC:  0.5337562656641603
               precision    recall  f1-score   support

No Recurrence       0.65      0.92      0.76       112
   Recurrence   

In [60]:
# predicting recurrence with absence/presence of 50 most common driver mutations

X_train, X_test, y_train, y_test = train_test_split(fifty_driver_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

lr_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(lr_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 50) (169, 50) (252,) (169,)


Best parameters:  {'C': 1e-05}
Best score:  0.5
Predictions:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5
ROC AUC:  0.4783051378446116
               precision    recall  f1-score   support

No Recurrence       0.66      1.00      0.80       112
   Recurrence       0.00      0.00      0.00        57

     accuracy        

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [65]:
# predicting recurrence with absence/presence of 20 most common driver mutations

X_train, X_test, y_train, y_test = train_test_split(twenty_driver_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

lr_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(lr_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 20) (169, 20) (252,) (169,)
Best parameters:  {'C': 1e-05}
Best score:  0.5
Predictions:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5
ROC AUC:  0.47352756892230574
               precision    recall  f1-score   support

No Recurrence       0.66      1.00      0.80       112
   Recurrence       0.00      0.00      0.

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [66]:
X_train, X_test, y_train, y_test = train_test_split(fifty_gene_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

svm_model = svm.SVC(probability=True)

grid_search = GridSearchCV(svm_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 50) (169, 50) (252,) (169,)


Best parameters:  {'C': 1e-05}
Best score:  0.5
Predictions:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5
ROC AUC:  0.5159774436090225
               precision    recall  f1-score   support

No Recurrence       0.66      1.00      0.80       112
   Recurrence       0.00      0.00      0.00        57

     accuracy        

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [67]:
X_train, X_test, y_train, y_test = train_test_split(twenty_gene_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

svm_model = svm.SVC(probability=True)

grid_search = GridSearchCV(svm_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 20) (169, 20) (252,) (169,)


Best parameters:  {'C': 10}
Best score:  0.5150623885918003
Predictions:  [0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 0 1 1 0 1 0 1 0 0
 0 0 0 0 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 1 1 0 0 1 0 0
 0 0 1 0 1 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0
 0 0 0 1 0 0 1 0 0 1 0 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0
 0 0 0 0 1 1 0 1 0 1 1 1 1 0 0 0 0 0 0 0 1]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5939849624060151
ROC AUC:  0.36928258145363413
               precision    recall  f1-score   support

No Recurrence       0.73      0.71      0.72       112
   Recurrence       0.46      0.47      0.47     

In [68]:
X_train, X_test, y_train, y_test = train_test_split(fifty_driver_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

svm_model = svm.SVC(probability=True)

grid_search = GridSearchCV(svm_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 50) (169, 50) (252,) (169,)


Best parameters:  {'C': 1e-05}
Best score:  0.5
Predictions:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5
ROC AUC:  0.5321898496240601
               precision    recall  f1-score   support

No Recurrence       0.66      1.00      0.80       112
   Recurrence       0.00      0.00      0.00        57

     accuracy        

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [69]:
X_train, X_test, y_train, y_test = train_test_split(twenty_driver_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

svm_model = svm.SVC(probability=True)

grid_search = GridSearchCV(svm_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 20) (169, 20) (252,) (169,)
Best parameters:  {'C': 10}
Best score:  0.5146167557932264
Predictions:  [0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 0
 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 1 0 1 1 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 1 1 1
 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 1]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.45825501253132833
ROC AUC:  0.44752506265664155
               precision    recall  f1-score   support

No Recurrence       0.64      0.74      0.69       112
   Recurrence 

In [71]:
# Neural network/Multi-layer perceptron (MLP) algorithm - layer sizes are optimized
X_train, X_test, y_train, y_test = train_test_split(fifty_gene_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 50) (169, 50) (252,) (169,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py

Best parameters:  {'alpha': 0.1, 'hidden_layer_sizes': (10,)}
Best score:  0.5647950089126559
Predictions:  [1 0 1 1 1 0 1 0 0 1 0 1 0 1 1 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 1 1 0 0 1 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 0 1 1 0 1 1 1 0 0 0 0 1 1 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 0 1 1 1 0 1 0 0 0 1
 0 1 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 0 1]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5236528822055138
ROC AUC:  0.5144110275689222
               precision    recall  f1-score   support

No Recurrence       0.68      0.70      0.69       112
   Recurrence   

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [72]:
# Neural network/Multi-layer perceptron (MLP) algorithm - layer sizes are optimized
X_train, X_test, y_train, y_test = train_test_split(twenty_gene_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 20) (169, 20) (252,) (169,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_

Best parameters:  {'alpha': 0.01, 'hidden_layer_sizes': (10,)}
Best score:  0.5238859180035651
Predictions:  [1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0
 0 0 0 0 1 1 1 0 1 0 1 1 0 0 0 0 0 0 1 1 1 0 1 0 0 1 0 0 0 1 1 0 0 0 1 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 1 1 0 1 0 0 0 0 0 0 1 1 1 0 1 1 0 1 1 1 0 0 0 0
 1 0 1 1 0 0 1 0 0 0 1 1 1 0 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1
 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 1]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5013314536340852
ROC AUC:  0.5384555137844612
               precision    recall  f1-score   support

No Recurrence       0.66      0.65      0.66       112
   Recurrence  

In [73]:
# Neural network/Multi-layer perceptron (MLP) algorithm - layer sizes are optimized
X_train, X_test, y_train, y_test = train_test_split(fifty_driver_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 50) (169, 50) (252,) (169,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_

Best parameters:  {'alpha': 0.1, 'hidden_layer_sizes': (10,)}
Best score:  0.5320855614973262
Predictions:  [0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 1 0 0 0 0 1 1 1 1 0 1 1 0 0 1 0 0 1 0 0 1
 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0
 1 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 1 1 1 0 0 0 0 0 1 0 1
 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 1
 0 0 0 1 1 0 1 0 1 1 0 1 1 1 1 0 0 1 1 1 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5013314536340852
ROC AUC:  0.4662437343358396
               precision    recall  f1-score   support

No Recurrence       0.66      0.65      0.66       112
   Recurrence   

In [74]:
# Neural network/Multi-layer perceptron (MLP) algorithm - layer sizes are optimized
X_train, X_test, y_train, y_test = train_test_split(twenty_driver_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 20) (169, 20) (252,) (169,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network

Best parameters:  {'alpha': 1e-05, 'hidden_layer_sizes': (5,)}
Best score:  0.5974153297682709
Predictions:  [0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 1 1 1 1 0 0 0 1 0 0 1
 1 0 0 0 1 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 1 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 0
 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 1 1 0 1 0 0 0 1 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 1 1
 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.4882518796992481
ROC AUC:  0.4660870927318296
               precision    recall  f1-score   support

No Recurrence       0.65      0.66      0.66       112
   Recurrence  

# Extract the gene names from the driver_frequency_list
ten_drivers = [gene for gene, _ in driver_frequency_list[:10]]

# Filter the original DataFrame based on the selected genes
filtered_drivers_table = mut_table[mut_table['Hugo_Symbol'].isin(ten_drivers)]

# Create a binary matrix based on both gene presence/absence and 'DriverMut' values
ten_driver_matrix = pd.crosstab(filtered_drivers_table['patient_id'], filtered_drivers_table['Hugo_Symbol'], values=filtered_drivers_table['DriverMut'].astype(int), aggfunc='max').clip(upper=1).fillna(0)

# Identify missing patients
all_patients = set(mut_table['patient_id'])
patients_with_genes = set(filtered_drivers_table['patient_id'])
missing_patients = all_patients - patients_with_genes

# Add missing patients to the binary matrix with all 0s
for missing_patient in missing_patients:
    ten_driver_matrix.loc[missing_patient, :] = 0

# Explicitly set the data type of the entire DataFrame to integers
ten_driver_matrix = ten_driver_matrix.astype(int)

# Display the resulting gene matrix
print(ten_driver_matrix.shape)
print(ten_driver_matrix)

ten_driver_matrix = ten_driver_matrix.sort_values(by=['patient_id'])
ten_driver_matrix.to_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/10_driver_matrix.csv')

In [75]:
# predicting recurrence with absence/presence of 10 most common driver mutations

X_train, X_test, y_train, y_test = train_test_split(ten_driver_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

lr_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(lr_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 10) (169, 10) (252,) (169,)
Best parameters:  {'C': 100}
Best score:  0.5137254901960784
Predictions:  [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.48598057644110276
ROC AUC:  0.4635808270676692
               precision    recall  f1-score   support

No Recurrence       0.66      0.90      0.76       112
   Recurrence 

In [76]:
X_train, X_test, y_train, y_test = train_test_split(ten_driver_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

svm_model = svm.SVC(probability=True)

grid_search = GridSearchCV(svm_model, param_grid, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 10) (169, 10) (252,) (169,)


Best parameters:  {'C': 1}
Best score:  0.5144385026737968
Predictions:  [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.4818295739348371
ROC AUC:  0.48222117794486213
               precision    recall  f1-score   support

No Recurrence       0.65      0.93      0.77       112
   Recurrence       0.20      0.04      0.06      

In [14]:
# Neural network/Multi-layer perceptron (MLP) algorithm - layer sizes are optimized
X_train, X_test, y_train, y_test = train_test_split(ten_driver_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 10) (169, 10) (252,) (169,)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Best parameters:  {'alpha': 1e-05, 'hidden_layer_sizes': (5,)}
Best score:  0.5552584670231729
Predictions:  [0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0
 0 0 1 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.4458020050125313
ROC AUC:  0.4295895989974937
               precision    recall  f1-score   support

No Recurrence       0.63      0.82      0.72       112
   Recurrence  

fifty_gene_matrix = fifty_gene_matrix.sort_values(by=['patient_id'])
fifty_driver_matrix = fifty_driver_matrix.sort_values(by=['patient_id'])
twenty_gene_matrix = twenty_gene_matrix.sort_values(by=['patient_id'])
twenty_driver_matrix = twenty_driver_matrix.sort_values(by=['patient_id'])

fifty_gene_matrix.to_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/50_gene_matrix.csv')
fifty_driver_matrix.to_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/50_driver_matrix.csv')
twenty_gene_matrix.to_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/20_gene_matrix.csv')
twenty_driver_matrix.to_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/20_driver_matrix.csv')

In [ ]:
logistic_100_10_balanced_accuracy = [0.52436, 0.52204, 0.55510, 0.48003, 0.45704, 0.46513, 0.51274]
svm_100_10_balanced_accuracy = [0.50017, 0.50017, 0.51274, 0.5, 0.49174, 0.47521, 0.50430]
neural_100_10_balanced_accuracy = [0.55346, 0.48778, 0.48717, 0.42820, 0.49578, 0.47598, 0.49638]

x = ['100 genes', '50 genes', '20 genes', '100 drivers', '50 drivers', '20 drivers', '10 drivers']
plt.plot(x, logistic_100_10_balanced_accuracy, label = "Logistic Regression", color = 'red')
plt.plot(x, svm_100_10_balanced_accuracy, label = "SVM", color = 'green')
plt.plot(x, neural_100_10_balanced_accuracy, label = "Neural Network", color = 'blue')
plt.xlabel('Number of genes/drivers (most common)')
plt.ylabel('Balanced Accuracy')
plt.ylim(0.4, 0.6)
plt.title('Balanced Accuracy for different number of genes/drivers using 3 different ML model types')
plt.legend()


In [ ]:
logistic_100_10_roc_auc = [0.52014, 0.54408, 0.52531, 0.49131, 0.46083, 0.47788, 0.48201]
svm_100_10_roc_auc = [0.52410, 0.54149, 0.38189, 0.47443, 0.48304, 0.45842, 0.50456]
neural_100_10_roc_auc = [0.51885, 0.48812, 0.47891, 0.43647, 0.49578, 0.47598, 0.44774]

x = ['100 genes', '50 genes', '20 genes', '100 drivers', '50 drivers', '20 drivers', '10 drivers']
plt.plot(x, logistic_100_10_roc_auc, label = "Logistic Regression", color = 'red')
plt.plot(x, svm_100_10_roc_auc, label = "SVM", color = 'green')
plt.plot(x, neural_100_10_roc_auc, label = "Neural Network", color = 'blue')
plt.xlabel('Number of genes/drivers (most common)')
plt.ylabel('ROC AUC')
plt.ylim(0.35, 0.6)
plt.title('ROC AUC for different number of genes/drivers using 3 different ML model types')
plt.legend()

In [ ]:
logistic_100_10_f1_recurrence = [0.35, 0.32, 0.35, 0.20, 0.13, 0.09, 0.14]
logistic_100_10_f1_no_recurrence = [0.69, 0.72, 0.76, 0.75, 0.75, 0.77, 0.82]
svm_100_10_f1_recurrence = [0.07, 0.07, 0.14, 0.00, 0.00, 0.00, 0.07]
svm_100_10_f1_no_recurrence = [0.82, 0.82, 0.82, 0.83, 0.83, 0.81, 0.82]
neural_100_10_f1_recurrence = [0.39, 0.07, 0.32, 0.35, 0.29, 0.20, 0.17]
neural_100_10_f1_no_recurrence = [0.71, 0.81, 0.64, 0.34, 0.68, 0.79, 0.78]

x = ['100 genes', '50 genes', '20 genes', '100 drivers', '50 drivers', '20 drivers', '10 drivers']
plt.plot(x, logistic_100_10_f1_recurrence, label = "Logistic Regression - Recurrence", color = 'red')
plt.plot(x, logistic_100_10_f1_no_recurrence, label = "Logistic Regression - No Recurrence", color = 'orange')
plt.plot(x, svm_100_10_f1_recurrence, label = "SVM - Recurrence", color = 'green')
plt.plot(x, svm_100_10_f1_no_recurrence, label = "SVM - No Recurrence", color = 'lightgreen')
plt.plot(x, neural_100_10_f1_recurrence, label = "Neural Network - Recurrence", color = 'blue')
plt.plot(x, neural_100_10_f1_no_recurrence, label = "Neural Network - No Recurrence", color = 'lightblue')
plt.xlabel('Number of genes/drivers (most common)')
plt.ylabel('F1 Score')
plt.ylim(0, 1)
plt.title('F1 Score for different number of genes/drivers using 3 different ML model types')
plt.legend(bbox_to_anchor = (1.05, 1.0), loc='upper left')

In [9]:
sum(i for i in y)/len(y)

0.33729216152019004